# <br><center> Air New Zealand's weekly stock price prediction based on Google Trend Indexes </center>

## Table of Contents
* [Content](#Table-of-Contents)
* [1. Introduction](#1.-Introduction)
* [2. Data Acquisition](#2.-Data-Acquisition)
    * [2.1 Google Top Trend](#2.1-Google-Top-Trend)
    * [2.2 Yahoo Finance](#2.2-Yahoo-Finance)
* [3. Data Analysis](#3.-Data-Analysis)
    * [3.1 Correlation Analysis](#3.1-Correlation-Analysis)
    * [3.2 Linear Analysis](#3.2-Linear-Analysis)
        * [Bivariate Analysis](#Bivariate-Analysis)
        * [Multivariate Analysis with current stock price](#Multivariate-Analysis-with-current-stock-price)
        * [Multivariate Analysis only using Google Trend Indexes](#Multivariate-Analysis-only-using-Google-Trend-Indexes)
        * [Multivariate Analysis using sklearn library](#Multivariate-Analysis-using-sklearn-library)
        * [Normalization using logarithm](#Normalization-using-logarithm)
        * [Linear Regression Models Comparison](#Linear-Regression-Models-Comparison)
        * [Normalization using minimum and maximum](#Normalization-using-minimum-and-maximum)
    * [3.3 Logistic Regression](#3.3-Logistic-Regression)
    * [3.4 KNN](#3.4-KNN)
* [4. Conclusion](#4.-Conclusion)

## 1. Introduction

This report finds the relationship between stock price of Air New Zealand and some keywords related to the company such as 'air new zealand', 'flight' and so on. Also, the report discovers prediction models.

In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import seaborn as sns
import time
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import statsmodels.formula.api as smf
import math 
from sklearn import neighbors
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import ensemble
from scipy.stats import skew
from scipy.stats import norm
from scipy.stats.stats import pearsonr
from scipy import stats
from pylab import rcParams
import mpld3

import tweepy
import csv
import os
import re

from pytrends.request import TrendReq
from yahoo_finance import Share
from datetime import date
import datetime as dt
import datetime
from dateutil.relativedelta import relativedelta

import warnings
warnings.filterwarnings('ignore')

## 2. Data Acquisition

### 2.1 Google Top Trend
This part loads the data using the API. Data is from five years ago to the present and weekly.(source: https://trends.google.co.nz/trends/)

**Key Words ** 

searched worldwide: air new zealand, auckland, wellington

searched in New Zealand: air new zealand, flight, travel, airpoints, grab a seat, check in, fiji, dunedin, christchurch, queenstown, nelson


In [2]:
# This is from https://github.com/GeneralMills/pytrends and is not an Google official API.
# The ID is a kind of dummy, not being used one.

pytrends = TrendReq('peterhwang105@gmail.com', 'Machine158.222', hl='en-US', tz=360, custom_useragent=None)

In [3]:
pytrends.build_payload(kw_list=['air new zealand'], cat=0, timeframe='today 5-y', geo='', gprop='')
name = pytrends.interest_over_time()
name = name.rename(columns={'air new zealand':'airNZ'})
pytrends.build_payload(kw_list=['air new zealand'], cat=0, timeframe='today 5-y', geo='NZ', gprop='')
namenz = pytrends.interest_over_time()
namenz = namenz.rename(columns={'air new zealand':'airNZ_NZ'})
pytrends.build_payload(kw_list=['flight'], cat=0, timeframe='today 5-y', geo='NZ', gprop='')
flight = pytrends.interest_over_time()
pytrends.build_payload(kw_list=['travel'], cat=0, timeframe='today 5-y', geo='NZ', gprop='')
travel = pytrends.interest_over_time()
pytrends.build_payload(kw_list=['airpoints'], cat=0, timeframe='today 5-y', geo='NZ', gprop='')
point = pytrends.interest_over_time()
pytrends.build_payload(kw_list=['grab a seat'], cat=0, timeframe='today 5-y', geo='NZ', gprop='')
grab = pytrends.interest_over_time()
grab = grab.rename(columns={'grab a seat':'grab_a_seat'})
pytrends.build_payload(kw_list=['check in'], cat=0, timeframe='today 5-y', geo='NZ', gprop='')
check = pytrends.interest_over_time()
check = check.rename(columns={'check in':'check_in'})
pytrends.build_payload(kw_list=['fiji'], cat=0, timeframe='today 5-y', geo='NZ', gprop='')
fiji = pytrends.interest_over_time()
pytrends.build_payload(kw_list=['auckland'], cat=0, timeframe='today 5-y', geo='', gprop='')
auckland = pytrends.interest_over_time()
pytrends.build_payload(kw_list=['wellington'], cat=0, timeframe='today 5-y', geo='', gprop='')
wellington = pytrends.interest_over_time()
pytrends.build_payload(kw_list=['dunedin'], cat=0, timeframe='today 5-y', geo='NZ', gprop='')
dunedin = pytrends.interest_over_time()
pytrends.build_payload(kw_list=['christchurch'], cat=0, timeframe='today 5-y', geo='NZ', gprop='')
church = pytrends.interest_over_time()
pytrends.build_payload(kw_list=['queenstown'], cat=0, timeframe='today 5-y', geo='NZ', gprop='')
queen = pytrends.interest_over_time()
pytrends.build_payload(kw_list=['nelson'], cat=0, timeframe='today 5-y', geo='NZ', gprop='')
nelson = pytrends.interest_over_time()
airGgl = pd.concat([name, namenz, flight, travel, point, grab, check, fiji, auckland, wellington, dunedin, church, queen, nelson], axis=1)

In [4]:
airGgl = airGgl.astype(float)
airGgl.head()

,airNZ,airNZ_NZ,flight,travel,airpoints,grab_a_seat,check_in,fiji,auckland,wellington,dunedin,christchurch,queenstown,nelson
date,,,,,,,,,,,,,,
2012-08-05,77.0,74.0,45.0,88.0,49.0,85.0,46.0,43.0,72.0,57.0,55.0,54.0,39.0,40.0
2012-08-12,78.0,78.0,47.0,89.0,43.0,79.0,39.0,52.0,75.0,58.0,57.0,57.0,44.0,43.0
2012-08-19,75.0,71.0,46.0,84.0,45.0,71.0,54.0,45.0,75.0,59.0,56.0,55.0,45.0,40.0
2012-08-26,75.0,71.0,52.0,79.0,40.0,83.0,56.0,47.0,71.0,58.0,55.0,55.0,40.0,41.0
2012-09-02,76.0,77.0,54.0,86.0,56.0,76.0,44.0,44.0,74.0,58.0,57.0,55.0,39.0,43.0


### 2.2 Yahoo Finance

In [5]:
#gain today and 5 years ago date
today=dt.date.today()
fiveYearAgo = today.replace(year = today.year -5)
#coverting datetime into string
today = today.strftime('%Y-%m-%d')
fiveYearAgo = fiveYearAgo.strftime('%Y-%m-%d')

In [6]:
# function define
# This function is to get average adjusted value, next week average value, volume of a week and the date(First date of the week which is Sunday)
def avrVolValNvalNDate(df):
    pre = 4
    aVol = []
    date = []
    aAdj = []
    nAdj = []
    vsum = 0
    asum = 0
    vcount = 0
    acount = 0
    nAV = np.nan
    for index, adj, vol in zip(df.index, df['Adj_Close'],df['Volume']):
        if(pre<index.weekday()):
            if(vcount==0):
                aVol.append(0)
            else:
                aVol.append((vsum/vcount))
            aAdj.append((asum/acount))
            nAdj.append(nAV)
            nAV = asum/acount
            if(index.weekday()==4):
                date.append(index + datetime.timedelta(days=2))
            elif(index.weekday()==3):
                date.append(index + datetime.timedelta(days=3))
            elif(index.weekday()==2):
                date.append(index + datetime.timedelta(days=4))
            elif(index.weekday()==1):
                date.append(index + datetime.timedelta(days=5))
            elif(index.weekday()==0):
                date.append(index + datetime.timedelta(days=6))
            pre = index.weekday()
            vsum = vol
            asum = adj
            vcount = 1
            if(vol==0):
                vcount = 0
            acount = 1
        else:
            vsum = vsum + vol
            asum = asum + adj
            acount = acount +1
            vcount = vcount +1
            if(vol==0):
                vcount = vcount -1 #volume 0 means that the market did not open on that day
            pre = index.weekday()
    return aVol, aAdj, nAdj, date

In [7]:
# this function adds a column which indicates whether the stock price increased or decreased
# 0 : same or decreased
# 1 : increased
def checkINC(df):
    temp = []
    for nV, cV in zip(df['Nxt_Value'],df['Value']):
        if((nV-cV) > 0.0):
            temp.append(1)
        elif((nV-cV) <= 0.0):
            temp.append(0)
    return temp

In [8]:
# this function is for gaining the change rate of index
def rate(df):
    d = pd.DataFrame()
    for col in df.columns[:]:
        pre = -1.0
        temp = []
        for row in df[col]:
            if(pre == -1.0):
                temp.append(np.nan)
                pre = float(row)
            else :
                row = float(row)
                r = ((row - pre)/pre)
                temp.append(r)
                pre = row
        d['r'+col] = temp
    d.index = df.index
    d = d.dropna(axis=0)
    return d

In [9]:
from yahoo_finance import Share

In [10]:
air_s = Share('AIR.NZ')

In [11]:
air_s.get_historical("2016-01-01", "2017-01-01")

YQLResponseMalformedError: Response malformed.

In [ ]:
air_history = air_s.get_historical(fiveYearAgo, today)

In [ ]:
air_history = pd.DataFrame(air_history)
air_history = air_history.set_index('Date')
air_history.index = pd.to_datetime(air_history.index)
air_history['Volume'] = pd.to_numeric(air_history['Volume'])
air_history['Adj_Close'] = pd.to_numeric(air_history['Adj_Close'])

In [ ]:
vol_temp = []
adj_temp = []
date_temp = []
nAdj_temp = []
vol_temp, adj_temp, nAdj_temp, date_temp = avrVolValNvalNDate(air_history)

In [ ]:
temp = []
temp = pd.DataFrame({'Volume':vol_temp,'Value':adj_temp, 'Nxt_Value': nAdj_temp}, index=date_temp)
air = pd.concat([temp, airGgl], axis=1, join_axes=[airGgl.index])
air = air.dropna(axis = 0)

airNZ_rate = rate(airGgl)
airNZ_rate = pd.concat([temp, airNZ_rate], axis=1, join_axes=[airNZ_rate.index])
airNZ_rate = airNZ_rate.dropna(axis = 0)

air['Check_INC'] = checkINC(air)
cols = air.columns.tolist()
cols = cols[-1:] + cols[:-1]
air = air[cols]

# to predict using the change rate of Google Trend Indexes
airNZ_rate['Check_INC'] = checkINC(airNZ_rate)
cols = airNZ_rate.columns.tolist()
cols = cols[-1:] + cols[:-1]
airNZ_rate = airNZ_rate[cols]

The Google Trend data and Stock data are kept changing, so the research uses the data on 21/4/2017 store into csv files. The main purpose is to remove outliers. Fron now on, the report finds several models based on the files.

In [ ]:
#air.to_csv('air.csv')
air = pd.read_csv('air.csv', index_col=0)

In [ ]:
air.head()

## 3. Data Analysis

### 3.1 Correlation Analysis

In [ ]:
# This table is based on Google Trend Indexes
air.corr().head()

In [ ]:
colormap = plt.cm.viridis
plt.figure(figsize=(14,14))
plt.title('Pearson Correlation of Air New Zealand stock price and volume and Google search', y=1.05, size=15)
sns.heatmap(air.astype(float).corr(),linewidths=0.1,vmax=1.0,  fmt='.2f', square=True, cmap='magma', linecolor='white', annot=True)

In [ ]:
# This table is to analyse based on the change rate of Google Trend Indexes
airNZ_rate.corr().head()

In [ ]:
colormap = plt.cm.viridis
plt.figure(figsize=(14,14))
plt.title('Pearson Correlation of Air New Zealand stock price and volume and Google search', y=1.05, size=15)
sns.heatmap(airNZ_rate.astype(float).corr(),linewidths=0.1,vmax=1.0,  fmt='.2f', square=True, cmap='magma', linecolor='white', annot=True)

The change rate of Google Trend Index seems not to have any relationship with stock price. So, the research only focuses on just Google Trend Index.

In [ ]:
corrmat = air.corr()
k = 18 #number of variables for heatmap
cols = corrmat.nlargest(k, 'Nxt_Value')['Nxt_Value'].index
cm = np.corrcoef(air[cols].values.T)
sns.set(font_scale=1)
plt.figure(figsize=(14,14))
hm = sns.heatmap(cm, cbar=True, linewidths=0.1,vmax=1.0,annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

We will use only variations which are correlation > |0.5|

In [ ]:
# COLUMNS: variations are correlation > |0.5|
COLUMNS = ['Nxt_Value', 'Value', 'check_in', 'wellington', 'queenstown', 'dunedin','airpoints', 'nelson', 'airNZ', 'grab_a_seat']

In [ ]:
sns.set()
sns.pairplot(air[COLUMNS], size = 2)
plt.show();

### 3.2 Linear Analysis

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

#### Remove Outliers

In [ ]:
var = 'Value'
data = pd.concat([air['Nxt_Value'], air[var]], axis=1)
data.plot.scatter(x=var, y='Nxt_Value')

In [ ]:
var = 'check_in'
data = pd.concat([air['Nxt_Value'], air[var]], axis=1)
data.plot.scatter(x=var, y='Nxt_Value')

In [ ]:
air.sort_values(by = var, ascending = False)[:3]

In [ ]:
air = air.drop(air[air[var] == 100.0].index)

In [ ]:
var = 'wellington'
data = pd.concat([air['Nxt_Value'], air[var]], axis=1)
data.plot.scatter(x=var, y='Nxt_Value')

In [ ]:
air.sort_values(by = var, ascending = False)[:18]

In [ ]:
# there are two points when wellington = 77.0 
# so remove the point using Volume
# make sure 276640.0 Volume is only one value.
air.Volume.value_counts()[276640.0]

In [ ]:
air = air.drop(air[air[var] == 100.0].index)
air = air.drop(air[air[var] == 93.0].index)
air = air.drop(air[air[var] == 92.0].index)
air = air.drop(air[air[var] == 91.0].index)
air = air.drop(air[air['Volume'] == 276640.0].index)

In [ ]:
var = 'queenstown'
data = pd.concat([air['Nxt_Value'], air[var]], axis=1)
data.plot.scatter(x=var, y='Nxt_Value')

In [ ]:
air.sort_values(by = var, ascending = False)[:8]

In [ ]:
# there are two points when queenstown = 80.0 
# so remove the point using Volume
# make sure 134700.0 Volume is only one value.
air.Volume.value_counts()[134700.0]

In [ ]:
air = air.drop(air[air[var] == 100.0].index)
air = air.drop(air[air[var] == 93.0].index)
air = air.drop(air[air[var] == 92.0].index)
air = air.drop(air[air[var] == 86.0].index)
air = air.drop(air[air['Volume'] == 134700.0].index)

In [ ]:
var = 'dunedin'
data = pd.concat([air['Nxt_Value'], air[var]], axis=1)
data.plot.scatter(x=var, y='Nxt_Value')

In [ ]:
air.sort_values(by = var, ascending = False)[:1]

In [ ]:
air = air.drop(air[air[var] == 100.0].index)

In [ ]:
var = 'airpoints'
data = pd.concat([air['Nxt_Value'], air[var]], axis=1)
data.plot.scatter(x=var, y='Nxt_Value')

In [ ]:
air.sort_values(by = var, ascending = False)[:3]

In [ ]:
air = air.drop(air[air[var] == 100.0].index)
air = air.drop(air[air[var] == 85.0].index)
air = air.drop(air[air[var] == 82.0].index)

In [ ]:
var = 'nelson'
data = pd.concat([air['Nxt_Value'], air[var]], axis=1)
data.plot.scatter(x=var, y='Nxt_Value')

In [ ]:
air.sort_values(by = var, ascending = False)[:3]

In [ ]:
air = air.drop(air[air[var] == 100.0].index)
air = air.drop(air[air[var] == 75.0].index)

In [ ]:
var = 'airNZ'
data = pd.concat([air['Nxt_Value'], air[var]], axis=1)
data.plot.scatter(x=var, y='Nxt_Value')

In [ ]:
var = 'grab_a_seat'
data = pd.concat([air['Nxt_Value'], air[var]], axis=1)
data.plot.scatter(x=var, y='Nxt_Value')

The points which grab a seat > 90 are obviously outliers. So get rid of them.

In [ ]:
air.sort_values(by = var, ascending = False)[:3]

In [ ]:
air = air.drop(air[air[var] == 100.0].index)
air = air.drop(air[air[var] == 94.0].index)

In [ ]:
corrmat = air.corr()
k = 18 #number of variables for heatmap
cols = corrmat.nlargest(k, 'Nxt_Value')['Nxt_Value'].index
cm = np.corrcoef(air[cols].values.T)
sns.set(font_scale=1)
plt.figure(figsize=(14,14))
hm = sns.heatmap(cm, cbar=True, linewidths=0.1,vmax=1.0,annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
sns.set()
sns.pairplot(air[COLUMNS], size = 2)
plt.show();

After remove outliers, some correlation values have been increased.

### Bivariate Analysis

In [ ]:
#generate the x-axis values that are in range for the 'Value' values
x = pd.DataFrame({'Value': np.linspace(air.Value.min(), air.Value.max(), len(air.Value))})

#generate the model which uses the stock price to predict the next stock price - the ols() return the generated model
mod = smf.ols(formula='Nxt_Value ~ 1 + Value', data=air).fit()

#plot the actual data
plt.scatter(air.Value, air.Nxt_Value, s=20, alpha=0.6)
plt.xlabel('Current Value'); plt.ylabel('Next Value')

#render the regression line by predicting the ys using the generated model from above
plt.plot(x.Value, mod.predict(x), 'b-', label='Linear $R^2$=%.2f' % mod.rsquared, alpha=0.9)

#give the figure a meaningful legend
plt.legend(loc='upper left', framealpha=0.5, prop={'size':'small'})
plt.title("Predicting the next week stock prices based on current week stock value")

#display the model statistics describing the goodness of fit
mod.summary()

In [ ]:
def gapPredicNReal(NxtV, Val):
    temp = []
    pred = []
    for nxt, crr in zip(NxtV, Val):
        est = crr*mod.params[1] +mod.params[0]
        gap = est - nxt
        pred.append(est)
        temp.append(gap)
    return temp, pred

In [ ]:
gapSngl, predicBi = gapPredicNReal(air['Nxt_Value'], air['Value'])

In [ ]:
x = air.index
y = gapSngl
x = pd.to_datetime(x)

trace1 = go.Scatter(
    x = x,
    y = y,
    name = 'Below',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(255, 182, 193, .9)',
        line = dict(
            width = 2,
        )
    )
)

data = [trace1]

layout = dict(title = 'Difference between forecasted and actual stock price',
              yaxis = dict(zeroline = True),
              xaxis = dict(zeroline = False)
             )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')


In [ ]:
snglErrors = []
print metrics.mean_absolute_error(air['Nxt_Value'], predicBi)
snglErrors.append(metrics.mean_absolute_error(air['Nxt_Value'], predicBi))
print metrics.mean_squared_error(air['Nxt_Value'], predicBi)
snglErrors.append(metrics.mean_squared_error(air['Nxt_Value'], predicBi))
print np.sqrt(metrics.mean_squared_error(air['Nxt_Value'], predicBi))
snglErrors.append(np.sqrt(metrics.mean_squared_error(air['Nxt_Value'], predicBi)))

### Multivariate Analysis with current stock price

In [ ]:
multi_linear = smf.ols(formula='Nxt_Value ~ 1 + Value + check_in + wellington + queenstown + dunedin + airpoints + nelson + airNZ + grab_a_seat', data=air[COLUMNS]).fit()
print multi_linear.params[0:10]
print 'R-Squared: ', multi_linear.rsquared
multi_linear.summary()

In [ ]:
def gapPredMultiVal(df): 
    temp = []
    pred = []
    for row in range(0,len(df)):
        estimate = 0.0
        for col in df.columns[1:]:
            estimate = estimate + (multi_linear.params[col])*df.ix[row,col]
        estimate = estimate +(multi_linear.params['Intercept'])
        gap = estimate - df.ix[row,'Nxt_Value']
        pred.append(estimate)
        temp.append(gap)
    return temp, pred

In [ ]:
gapMulti, predicMlt = gapPredMultiVal(air[COLUMNS])

In [ ]:
x = np.array(air.index)
y = gapMulti
x = pd.to_datetime(x)

trace1 = go.Scatter(
    x = x,
    y = y,
    name = 'Below',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(255, 182, 193, .9)',
        line = dict(
            width = 2,
        )
    )
)

data = [trace1]

layout = dict(title = 'Difference between forecasted and actual stock price',
              yaxis = dict(zeroline = True),
              xaxis = dict(zeroline = False)
             )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')

In [ ]:
multiErrors = []
print metrics.mean_absolute_error(air['Nxt_Value'], predicMlt)
multiErrors.append(metrics.mean_absolute_error(air['Nxt_Value'], predicMlt))
print metrics.mean_squared_error(air['Nxt_Value'], predicMlt)
multiErrors.append(metrics.mean_squared_error(air['Nxt_Value'], predicMlt))
print np.sqrt(metrics.mean_squared_error(air['Nxt_Value'], predicMlt))
multiErrors.append(np.sqrt(metrics.mean_squared_error(air['Nxt_Value'], predicMlt)))

### Multivariate Analysis only using Google Trend Indexes

In [ ]:
multi_linear = smf.ols(formula='Nxt_Value ~ 1 + check_in + wellington + queenstown + dunedin + airpoints + nelson + airNZ + grab_a_seat', data=air[COLUMNS]).fit()
print multi_linear.params[0:10]
print 'R-Squared: ', multi_linear.rsquared
multi_linear.summary()

In [ ]:
tempCols = ['Nxt_Value','check_in','wellington','queenstown','dunedin','airpoints','nelson','airNZ','grab_a_seat']

In [ ]:
gapOnlyGTrend, predicOnlyGg = gapPredMultiVal(air[tempCols])

In [ ]:
x = np.array(air.index)
y = gapOnlyGTrend
x = pd.to_datetime(x)

trace1 = go.Scatter(
    x = x,
    y = y,
    name = 'Below',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(255, 182, 193, .9)',
        line = dict(
            width = 2,
        )
    )
)

data = [trace1]

layout = dict(title = 'Difference between forecasted and actual stock price',
              yaxis = dict(zeroline = True),
              xaxis = dict(zeroline = False)
             )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')

In [ ]:
multiGgErrors = []
print metrics.mean_absolute_error(air['Nxt_Value'], predicOnlyGg)
multiGgErrors.append(metrics.mean_absolute_error(air['Nxt_Value'], predicOnlyGg))
print metrics.mean_squared_error(air['Nxt_Value'], predicOnlyGg)
multiGgErrors.append(metrics.mean_squared_error(air['Nxt_Value'], predicOnlyGg))
print np.sqrt(metrics.mean_squared_error(air['Nxt_Value'], predicOnlyGg))
multiGgErrors.append(np.sqrt(metrics.mean_squared_error(air['Nxt_Value'], predicOnlyGg)))

### Multivariate Analysis using sklearn library

In [ ]:
X = air[COLUMNS[1:]]
y = air['Nxt_Value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101)

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)
print(lm.intercept_)
print(lm.coef_)

In [ ]:
cdf = pd.DataFrame(lm.coef_, X.columns, columns=['Coeff'])
predictions = lm.predict(X_test)

In [ ]:
plt.scatter(y_test, predictions)

In [ ]:
d = pd.to_datetime(y_test.index)
trace1 = go.Scatter(
    x = d,
    y = (y_test-predictions),
    name = 'Below',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(255, 182, 193, .9)',
        line = dict(
            width = 2,
        )
    )
)

data = [trace1]

layout = dict(title = 'Difference between forecasted and actual stock price',
              yaxis = dict(zeroline = True),
              xaxis = dict(zeroline = False)
             )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')

In [ ]:
print("Skewness: %f" % (y_test-predictions).skew())
sns.distplot((y_test-predictions), fit=norm)
plt.xlabel('y_test-predictions')

In [ ]:
sklearnErrors = []
print metrics.mean_absolute_error(y_test, predictions)
sklearnErrors.append(metrics.mean_absolute_error(y_test, predictions))
print metrics.mean_squared_error(y_test, predictions)
sklearnErrors.append(metrics.mean_squared_error(y_test, predictions))
print np.sqrt(metrics.mean_squared_error(y_test, predictions))
sklearnErrors.append(np.sqrt(metrics.mean_squared_error(y_test, predictions)))

### Normalization using logarithm

In [ ]:
#histogram and normal probability plot
#skewness and kurtosis
print("Skewness: %f" % air['Nxt_Value'].skew())
print("Kurtosis: %f" % air['Nxt_Value'].kurt())
sns.distplot(air['Nxt_Value'], fit=norm);
fig = plt.figure()
res = stats.probplot(air['Nxt_Value'], plot=plt)

In [ ]:
#applying log transformation
air_log = pd.DataFrame()
for col in air.columns[1:]:
    air_log.loc[:,col] = np.log(air.loc[:,col])

In [ ]:
print("Skewness: %f" % air_log['Nxt_Value'].skew())
print("Kurtosis: %f" % air_log['Nxt_Value'].kurt())
sns.distplot(air_log['Nxt_Value'], fit=norm);
fig = plt.figure()
res = stats.probplot(air_log['Nxt_Value'], plot=plt)

In [ ]:
sns.set()
sns.pairplot(air_log[COLUMNS], size = 2)
plt.show();

Most of the data has changed to more normal. But, the absolute skewness value of the next stock prices as a predictor has been increased. So, this research did not use logarithm.

In [ ]:
print 'Original correlation:',(air[air.columns[:]].corr()['Volume']['Nxt_Value'])
print 'After transformed:',(air_log[air_log.columns[:]].corr()['Volume']['Nxt_Value'])

After transformed, the research found that the correlationship between 'Volume' and 'Nxt_Value' has significantly increased from -0.000116629340474 to 0.487494271303. But it is still less than 0.5 so we do not use 'Volume'.

In [ ]:
X = air_log[COLUMNS[1:]]
y = air_log['Nxt_Value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101)

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)
print(lm.intercept_)
print(lm.coef_)

In [ ]:
cdf = pd.DataFrame(lm.coef_, X.columns, columns=['Coeff'])
predictions = lm.predict(X_test)

In [ ]:
plt.scatter(np.exp(y_test), np.exp(predictions))

In [ ]:
d = pd.to_datetime(y_test.index)
trace1 = go.Scatter(
    x = d,
    y = (np.exp(y_test)-np.exp(predictions)),
    name = 'Below',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(255, 182, 193, .9)',
        line = dict(
            width = 2,
        )
    )
)

data = [trace1]

layout = dict(title = 'Difference between forecasted and actual stock price',
              yaxis = dict(zeroline = True),
              xaxis = dict(zeroline = False)
             )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')

In [ ]:
logErrors = []
print metrics.mean_absolute_error(np.exp(y_test), np.exp(predictions))
logErrors.append(metrics.mean_absolute_error(np.exp(y_test), np.exp(predictions)))
print metrics.mean_squared_error(np.exp(y_test), np.exp(predictions))
logErrors.append(metrics.mean_squared_error(np.exp(y_test), np.exp(predictions)))
print np.sqrt(metrics.mean_squared_error(np.exp(y_test), np.exp(predictions)))
logErrors.append(np.sqrt(metrics.mean_squared_error(np.exp(y_test), np.exp(predictions))))

### Linear Regression Models Comparison

In [ ]:
print 'Bivariate:                        ', snglErrors
print 'Multi-variate with stock:         ', multiErrors
print 'Multi-variate only Google Indexes:', multiGgErrors
print 'Multi-variate using sklearn lib:  ', sklearnErrors
print 'Multi-variate after log transform:', logErrors

Amongst 4 models, the model using sklearn library has the lowest error values. This model may be the best model.

### Normalization using minimum and maximum

In [ ]:
minmax_scale = preprocessing.MinMaxScaler().fit(air[COLUMNS])
air_minmax = minmax_scale.transform(air[COLUMNS])
air_minmax = pd.DataFrame(air_minmax)

In [ ]:
print("Skewness: %f" % air_minmax[0].skew())
print("Kurtosis: %f" % air_minmax[0].kurt())
sns.distplot(air_minmax[0], fit=norm);
fig = plt.figure()
res = stats.probplot(air_minmax[0], plot=plt)

Normalization using minimum and maximum seems not good since the skewness has not changed significantly.

### 3.3 Logistic Regression

This model predict whether the next week's average stock price will be increased or decreased.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
X = air[COLUMNS[1:]]
y = air['Check_INC']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101)

In [ ]:
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

In [ ]:
predictions = logmodel.predict(X_test)

In [ ]:
print(classification_report(y_test,predictions))

Not so bad. This model has a probability of 75%.

### 3.4 KNN

This section predicts that the next week average stock value will be increased or decreased using KNN model.

In [ ]:
scaler = StandardScaler()
scaler.fit(air[COLUMNS[1:]])
scaled_features = scaler.transform(air[COLUMNS[1:]])

In [ ]:
air_feat = pd.DataFrame(scaled_features, columns=COLUMNS[1:])

In [ ]:
air_feat.head()

In [ ]:
X = air_feat
y = air['Check_INC']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
pred = knn.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test,pred))

In [ ]:
error_rate = []

for i in range(1,40):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred_i = knn.predict(X_test)
    error_rate.append(np.mean(pred_i != y_test))

In [ ]:
plt.figure(figsize = (10,6))
plt.plot(range(1,40),error_rate, color='blue', linestyle='dashed', marker='o', markerfacecolor='red', markersize = 8)
plt.title('Error Rate vs K value')
plt.xlabel('K')
plt.ylabel('Error Rate')

In [ ]:
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train, y_train)
pred = knn.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test,pred))

This KNN model seems worse than logistic regression model. It has a probabilty of 67%.

### 4. Conclusion

Although stock and google index have relatively high correlation, it is only a probabilty of 75% to predict that the stock would go up or down. But, as linear models showed, Google Trend Index may be used as an auxiliary measure to predict stock price.

This part is added.

In [ ]:
air.head()

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
X = air.drop('Check_INC', axis=1)
y = air['Check_INC']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)
predictions = dtree.predict(X_test)

In [ ]:
print (confusion_matrix(y_test, predictions))
print '\n'
print (classification_report(y_test, predictions))

#### Boosting Model

In [ ]:
air.head()

In [ ]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

In [ ]:
X = air.drop(['Check_INC','Nxt_Value'], axis=1)
y = air['Nxt_Value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
GBest = ensemble.GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=3, max_features='sqrt',
                                               min_samples_leaf=15, min_samples_split=10, loss='huber').fit(X, y)

In [ ]:
def logToReal(array):
    temp = []
    for log_price in array:
        temp.append(np.exp(log_price))
    return temp

In [ ]:
predictGB = GBest.fit(X, y).predict(X)
#predictGB_Val = logToReal(predictGB2)

In [ ]:
print metrics.mean_absolute_error(air['Nxt_Value'], predictGB)
print metrics.mean_squared_error(air['Nxt_Value'], predictGB)
print np.sqrt(metrics.mean_squared_error(air['Nxt_Value'], predictGB))

In [ ]:
trace0 = go.Bar(x=air.index,y=air['Nxt_Value'], name='actual')
trace1 = go.Bar(x=air.index, y=predictGB, name='predicted')
data = [trace0, trace1]
layout = go.Layout(
    xaxis=dict(tickangle=-45),
    barmode='group',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='angled-text-bar')

#### Prediction without current stock prices

In [ ]:
air.head()

In [ ]:
X = air.drop(['Check_INC','Nxt_Value','Value' ], axis=1)
y = air['Nxt_Value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
GBest = ensemble.GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=3, max_features='sqrt',
                                               min_samples_leaf=15, min_samples_split=10, loss='huber').fit(X, y)

In [ ]:
predictGB2 = GBest.fit(X, y).predict(X)

In [ ]:
print metrics.mean_absolute_error(air['Nxt_Value'], predictGB2)
print metrics.mean_squared_error(air['Nxt_Value'], predictGB2)
print np.sqrt(metrics.mean_squared_error(air['Nxt_Value'], predictGB2))

In [ ]:
trace0 = go.Bar(x=air.index,y=air['Nxt_Value'], name='actual')
trace1 = go.Bar(x=air.index, y=predictGB2, name='predicted')
data = [trace0, trace1]
layout = go.Layout(
    xaxis=dict(tickangle=-45),
    barmode='group',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='angled-text-bar')